In [2]:
import torch
import torch.nn as nn

In [4]:
vector_dimension = 768 

linear_dimension = 3072 # dimension of the linear layer of feed forward for normalization

image_size = 224 # pixels in each dimension of image

num_channels = 3 # number of channels in image (R, G, B)

patch_size = 16 # number of patches/ blocks we break each image into

attention_heads = 12 # number of attention heads

num_layers = 12 # number of transformer block layers in the model

dropout = 0.4

normalization_constant = 1e-6 # proportionality constant for normalization layer

num_image_tokens : int = None # number of tokens produced for each image i.e. it produces a list of vectors/ embeddings for a patch of each image

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

In [ ]:
# class for the embedding layer of the vision transformer model

class VisionEmbedding(nn.Module):

    def __init__(self, vector_dimension, patch_size, num_channels, image_size):
        super().__init__()

        self.convolutor = nn.Conv2d(
            in_channels = num_channels,  # 3 channels per image R, G, B
            out_channels= vector_dimension, 
            kernel_size= patch_size,
            stride = patch_size
            )
        
        self.num_patches = (image_size // patch_size) ** 2

        self.positional_embedding = nn.Embedding(self.num_patches, vector_dimension)

        self.register_buffer("positions", torch.arange(self.num_patches).expand((1, -1)), persistent= False)

    def forward(self, image_tensors):

        # image tensors are of shape [ batch_size x num_channels x height x width ] 

        # converting them to tensors of shape [ batch_size x vector_dimension x (height / patch_size) x (width / patch_size)]
        # using the CNN

        image_embeddings = self.convolutor(image_tensors) # [ batch_size x vector_dimension x (height / patch_size) x (width / patch_size) ]

        image_embeddings = image_embeddings.flatten(2) # [ batch_size x vector_dimension x num_patches ]

        image_embeddings = image_embeddings.transpose(1, 2) # [ batch_size x num_patches x vector_dimension ]

        return image_embeddings + self.positional_embedding(self.positions)

In [ ]:
# class for the Vision Encoder block MLP layer

class VisionMLP(nn.Module):

    def __init__(self, vector_dimension, linear_dimension):
        super().__init__()

        self.layer1 = nn.Linear(vector_dimension, linear_dimension)
        self.layer2 = nn.Linear(linear_dimension, vector_dimension)

    def forward(self, hidden_state):

        hidden_state = self.layer1(hidden_state)

        hidden_state = f.gelu(hidden_state, approximate= "tanh")

        return self.layer2(hidden_state)


In [ ]:
# class for the attention Block. Since it is an image transformer and not a langugae one, there is no need of a causal mask

class VisionAttention(nn.Module):

    def __init__(self, vector_dimension, attention_heads, dropout):
        super().__init__()

        self.num_heads = attention_heads
        self.dropout = nn.Dropout(dropout)
        self.vector_dimension = vector_dimension
        self.head_dimension = vector_dimension // attention_heads

        self.wQ = nn.Linear(vector_dimension, vector_dimension) # Query projection layer
        self.wK = nn.Linear(vector_dimension, vector_dimension) # Key projection layer
        self.wV = nn.Linear(vector_dimension, vector_dimension) # Value projection layer
        self.wO = nn.Linear(vector_dimension, vector_dimension) # Output projection layer

        self.scale = vector_dimension ** -0.5

    def forward(self, hidden_state):

        # hidden_state [ batch_size x num_patches x vector_dimension ]
        batch_size, sequence_len = hidden_state.size() # sequence len = num_patches

        queryProjection = self.wQ(hidden_state) # [ batch_size x num_patches x vector_dimension ]
        keyProjection = self.wQ(hidden_state) # [ batch_size x num_patches x vector_dimension ]
        valueProjection = self.wQ(hidden_state) # [ batch_size x num_patches x vector_dimension ]

        queryProjection = queryProjection.view(batch_size, sequence_len, self.num_heads, self.head_dimension) # [batch_size x sequence_len x num_heads x head_dimension]
        keyProjection = keyProjection.view(batch_size, sequence_len, self.num_heads, self.head_dimension) # [batch_size x sequence_len x num_heads x head_dimension]
        valueProjection = valueProjection.view(batch_size, sequence_len, self.num_heads, self.head_dimension) # [batch_size x sequence_len x num_heads x head_dimension]

        queryProjection = queryProjection.transpose(1, 2) # [batch_size x num_heads x sequence_len x head_dimension]
        keyProjection = keyProjection.transpose(1, 2) # [batch_size x num_heads x sequence_len x head_dimension]
        valueProjection = valueProjection.transpose(1, 2) # [batch_size x num_heads x sequence_len x head_dimension]

        attention_scores = torch.matmul(queryProjection, keyProjection.transpose(2, 3)) * self.scale # [batch_size x num_heads x sequence_len x sequence_len]

        attention_scores = f.softmax(attention_scores, dim = -1)

        attention_scores = torch.matmul(attention_scores, valueProjection) # [ batch_size x num_heads x  sequence_len x head_dimension]

        attention_scores = attention_scores.transpose(1, 2).contiguous() # [batch_size x sequence_len x num_heads x head_dimension]

        attention_scores = attention_scores.reshape(batch_size, sequence_len, self.vector_dimension) # [batch_size, sequence_len, vector_dimension]

        return self.wO(attention_scores)


In [ ]:
# class for a single encoder block of the vision transformer

class VisionEncoderBlock(nn.Module):

    def __init__(self, ):
        super().__init__(vector_dimension, normalization_constant, attention_heads, dropout, linear_dimension)

        self.layerNorm1 = nn.LayerNorm(vector_dimension, normalization_constant)
        self.layerNorm2 = nn.LayerNorm(vector_dimension, normalization_constant) 

        self.attentionBlock

        self.MLP 

    def forward(self, hidden_states):

        skip_connector = torch.clone(hidden_states)

        hidden_states = self.layerNorm1(hidden_states)

        hidden_states = self.attentionBlock()

        hidden_states = hidden_states + skip_connector

        skip_connector = hidden_states

        hidden_states = self.layerNorm2(hidden_states)

        return hidden_states + skip_connector


In [ ]:
class VisionTransformer(nn.Module):

    def __init__(self, vector_dimension, normalization_constant, patch_size, num_channels, image_size):
        super().__init__()

        self.embedding_layer = VisionEmbedding(vector_dimension, patch_size, num_channels, image_size)

        self.encoder_layer

        self.post_normalization_layer = nn.LayerNorm(vector_dimension, normalization_constant)

    def forward(self, image_data):

        hidden_state = self.embedding_layer(image_data)

        hidden_state = self.encoder_layer(hidden_state)

        hidden_state = self.post_normalization_layer(hidden_state)

        return hidden_state

NameError: name 'nn' is not defined